# landscape classification

In this case study, we will be exploring the use of convolution neural networks to classify images of landscapes. Identifying landsacpe or vegetation or land-use types from image data has important applications in agriculture and natural-resource management.

In this example, we'll be using a data set of images from the following possible types or classes:

* buildings representing cities or other human habitations
* forests
* glacier or ice-covered landscapes
* mountains
* sea or ocean
* streets or paved areas

I've packaged the image data into separate training and validation sub-sets, available for downoad at zenodo.

The following code cell will download the image data sets, confirm the number of images in each sub-set, and package the data sets into tensorflow Dataset objects for neural network training.

In [ ]:
import tensorflow as tf
import pathlib

# download training data
train_url = 'https://zenodo.org/record/5512793/files/train.tgz'
train_dir = tf.keras.utils.get_file(origin=train_url, fname='train', untar=True)
train_dir = pathlib.Path(train_dir)

# download validation data
valid_url = 'https://zenodo.org/record/5512793/files/valid.tgz'
valid_dir = tf.keras.utils.get_file(origin=valid_url, fname='valid', untar=True)
valid_dir = pathlib.Path(valid_dir)

# print number of training and validation images
train_image_count = len(list(train_dir.glob('*/*.jpg')))
valid_image_count = len(list(valid_dir.glob('*/*.jpg')))
print(train_image_count, valid_image_count)

# package images into tensorflow dataset objects
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=(150,150),
                                                                 batch_size=32)
valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,
                                                                 image_size=(150,150),
                                                                 batch_size=32)
# print tensorflow dataset objects
print(train_data, valid_data)

It should take a few seconds to download the data sets.

You should see that there are 14,034 total image files in the training data sub-set, and 3000 images in the validation sub-set. In both cases, there are 6 possible classes or landscape types.

Notice that the shape of the training and validation Dataset objects is the same:

    ((None, 150, 150, 3), (None, ))

That is, the image data (ignoring the batch dimension of None) consists of 150x150 pixel images with 3 color channels (ie, typical RGB image data). The labels are integer-valued class labels, which is pretty standard for image classification problems.

We'll need to remember that the images are 150x150x3, so we can specify the correct input shape for our neural network.

Also, we'll need to use SparseCategoricalCrossentropy loss when we fit our model to the training data, because the category labels are not one-hot encoded.

To start off, we'll build a very simple convolution neural network consisting of a single convolution layer with a single 3x3 filter and ReLU activation.

As a 'trick', we're going to automatically rescale our image data to be on the [0,1] scale *automatically* in our neural network. We'll do this by specifying a tf.keras.layers.experimental.preprocessing.Rescaling layer as the first layer in the network.

Because our image data has pixel values between 0 and 255, we'll need to 'rescale' them by a factor of:

    1.0/255

which we specify as the scaling factor for the Rescaling layer. We'll also need to specify the input shape of the network when we create the Rescaling layer, because it's the first layer in the network.

After 'flattening' the output of the convolution layer, we create a Dense output layer with 6 units (because there are 6 possible landscape classes), and softmax activation.

We're going to opt for the Adam optimizer in this case, as it will help our model fit run faster (ie, fewer epochs). With this much data, we don't want to wait around for the slower SGD optimzer to reach a good model fit. The Adam optimizer is typically 'faster' than SGD, and it has been widely used for training image classification networks.

Make sure we specify SparseCategoricalCrossentropy loss, record the model's accuracy as it trains, and we'll train for 20 epochs.

Make sure you use GPU resources for this run, or it will take a *long* time! Click on the downward-facing arrow in the upper right corner of colab, select "View resources", and then click "Change Runtime Type". Select "GPU" from the "Hardware acceleration" drop-down, and save. Now your model fit will run on a GPU.

In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

This model has 131,458 trainable parameters, nearly all of them in the Dense output layer.

You'll notice that, with the Adam optimizer, the model reaches >0.95 accuracy on the *training* data after only a few epochs of training. But, the accuracy on the *validation* data stays *very* low (around 0.38 in my case)!

There is clearly an 'overfitting' problem. This makes sense, given that we have 14,034 training images and 131,458 model parameters!

Let's try adding a Dropout layer to reduce overfitting.

In the following code cell, we remove 90% of the outputs from the convolution layer, before flattening the data and sending it to the Dense output layer.

In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dropout(rate=0.9))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

Well, we appear to have alleviated model overfitting to the training data; the model's accuracy on the training and validation data sub-sets is much more similar.

But, accuracy is pretty *low*, overall. In my case, I achieved a final model accuracy of 0.49 on the training data and 0.50 on the validation data. So, about 50% of the images are being correctly classified, but at least our model isn't overfitting.

Let's see if we can improve model accuracy, without exacerbating overfitting.

The first thing we'll try is to add more 'filters' to the convolution layer.

Try setting the number of filters to 32, rather than using a single filter, and see if that improves accuracy on both the training and validation data. You can edit the following code cell to increase the number of convolution filters.

There's no 'magic' number of convolution filters that works in all situations, and I'm not aware of any reasonable 'formula' for deriving an 'optimal' number of convolution filters for a specific problem. Rather, most netowork designers will choose a 'convenient' number of convolution filters, based loosely on what has been used successfully in the past. Commonly-used values are typically in the range of 32-128, although sometimes you'll see more than 128 filters for some large-scale image analysis problems. And yes, the number of filters is typically a power of 2.

In our case, we chose 32 filters, based on the following 'intuition'.

* 16 filters might be 'too small' to capture a sufficient number of 'features' in the image data, and
* much more than 32 filters might increase the parameter count in our model 'too much', given the relatively small amount of training data.

In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=FIXME, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dropout(rate=0.9))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

Well, now we've got the accuracy on the training data back up over 0.95, but the validation accuracy is still suffering a bit. In my case, it was 0.67 after 20 epochs of training.

We could try increasing the dropout rate in our Dropout layer, to try to reduce the observed overfitting, but this would likely negatively impact our training accuracy.

As an alternative, we'll use a type of neural network layer widely used in convolution networks called a MaxPooling layer.

Briefly, "MaxPooling" looks across a contiguous block of inputs and outputs the maximum value within that block. It's similar to taking an 'average' of a bunch of inputs, but instead of averaging all the values, it just takes the maximum over all inputs in the block.

Conceptually, MaxPooling is similar to a convolution: both methods consider a contiguous 'block' within a larger input 'image' and produce an output that is dependent on the inputs within the 'block'. Unlike a convolution layer, however, MaxPooling layers have *no* trainable parameters - they simply transmit the maximum value across a block of inputs.

So, MaxPooling can be used to 'decrease' the 'size' of 'image' data, without adding trainable parameters to the model.

MaxPooling layers are often used in convolution networks precisely for the purpose of 'compressing' the image data, without losing the main 'features' of the data, and without requiring any trainable parameters.

Tensorflow implements 2-dimensional MaxPooling as a tf.keras.layers.MaxPool2D object, which can be added to a neural network just like any other type of layer. By default, MaxPool2D objects 'pool' information from a 2x2 contiguous 'block' of inputs, so they decrease the image 'size' by a factor of 2 in both height and width. 

We can create a MaxPool2D object and add it to our model using the python code:

    model.add(tf.keras.layers.MaxPool2D())

Try adding a MaxPool2D layer to the model immediately following the convolution layer but before the dropout layer. Make sure you use 32 filters in the convolution layer!

In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=FIXME, kernel_size=(3,3), activation=tf.keras.activations.relu))
FIXME - add MaxPool2D layer
model.add(tf.keras.layers.Dropout(rate=0.9))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

Well, accuracy on the *training* data appears to have declined a bit (in my case, final training accuracy after 20 epochs was 0.85), and validation accuracy actually *increased* a tad (0.74, in my case).

So, we're 'moving' in the 'right' directions.

If we take a 'step back' and consider the *structure* of our little convolution network, we can see that we've created a 'block' of network layers consisting of the following:

1. a convolution layer with 32 filters
2. a MaxPooling layer
3. a dropout layer (with dropout rate 0.9)

Right now, the output from this 'block' is flattened and then sent to the 'decision' or 'classification' layer (ie, the output layer that actually 'classifies' the images).

However, we could consider 'replicating' this 'block' of 3 layers multiple times, in order to create a 'deeper' modular network. Many very deep neural networks are built this way: they are composed of a sequence of replicated 'blocks'. In our case, we could 'replicate' our convolution-pooling-dropout 'block' many times, transforming:

    rescaling
    convolution
    pooling
    dropout
    flatten
    output

into:

    rescaling
    convolution
    pooling
    dropout
    convolution
    pooling
    dropout
    convolution
    pooling
    dropout
    ...
    flatten
    output

Technically, there's nothing preventing us from replicating the convolution-pooling-dropout 'module' as many times as we want, although we'd want to keep the model's parameter count relatively low. Also, as both the convolution and pooling layers decrease the height and width of the 'image' data, at some point we'd wind up with a 1x1 'image' (although we could use 'same' padding to prevent this).

For now, let's try implementing a model with 3 convolution-pooling-dropout modules. Make sure you use 32 3x3 convolution filters in each of the convolution layers, and use 0.9 as the dropout rate in each of the dropout layers.

In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))

FIXME - build a 4-module convolution network

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

Did you notice that your 'deeper' network actually has *fewer* parameters!

This is because the vast majority of the trainable parmaeters typically comes from the final Dense output layer's connections. By reducing the size of the 'image' data (in this case, the final 'image' is 7x7), we've *dramatically* reduced the number of trainable parameters in the output layer.

But wow, our network is *not* performing very well. After 20 epochs of training, my network achieved only 0.49 accuracy on the training data, and the accuracy on the validation data was 18.

By 'compressing' the image data through multiple rounds of MaxPooling *and* using a *very* strong dropout rate after *every* convolution layer, we've effectively crippled our neural network's capacity!

Let's try *removing* all those Dropout layers, and see how the network behaves!

In [ ]:
# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))

FIXME - build a 4-module convolution network without any Dropout layers

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

Our model has essentially the same number of trainable parameters with or without the Dropout layers, but removing Dropout definitely *improved* accuracy on both training and validation data.

When trying to design a convolution network from scratch, it can be challenging to make reliable design decisions. Often times, network 'design' becomes a fairly haphazard 'random walk' through network 'space' - you try something, see how it works, and then try something else. Often times, strategies that improve accuracy on their own don't necessarily 'play well' together, so you can take a few steps forward, followed by a giant leap *backward* as you try to design a reliable neural network.

## transfer learning

An alternative to building a neural network 'from scratch' is to use a pre-existing neural network and then 'tweak' it to solve your specific problem.

One of the main ways to do this is called "transfer learning".

XX